In [1]:
import pandas as pd
import tensorflow as tf
from keras.layers import Dropout

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM

colnames = ['frequency', 'year', 'month', 'day',
'3시간기온_4', '3시간기온_7', '3시간기온_10', '3시간기온_13', '3시간기온_16', '3시간기온_19',
'3시간기온_22', '3시간기온_25', '3시간기온_28', '3시간기온_31', '3시간기온_34', '3시간기온_37',
'3시간기온_40', '3시간기온_43', '3시간기온_46', '3시간기온_49', '3시간기온_52', '3시간기온_55',
'강수형태_4', '강수형태_7', '강수형태_10', '강수형태_13', '강수형태_16', '강수형태_19',
'강수형태_22', '강수형태_25', '강수형태_28', '강수형태_31', '강수형태_34', '강수형태_37',
'강수형태_40', '강수형태_43', '강수형태_46', '강수형태_49', '강수형태_52', '강수형태_55',
'강수확률_4', '강수확률_7', '강수확률_10', '강수확률_13', '강수확률_16', '강수확률_19',
'강수확률_22', '강수확률_25', '강수확률_28', '강수확률_31', '강수확률_34', '강수확률_37',
'강수확률_40', '강수확률_43', '강수확률_46', '강수확률_49', '강수확률_52', '강수확률_55',
'습도_4', '습도_7', '습도_10', '습도_13', '습도_16', '습도_19',
'습도_22', '습도_25', '습도_28', '습도_31', '습도_34', '습도_37',
'습도_40', '습도_43', '습도_46', '습도_49', '습도_52', '습도_55',
'일최고기온_4', '일최고기온_7', '일최저기온_4', '일최저기온_7',
'pop_ratio', 'man_pop', 'women_pop',
 'man_0~4', 'man_5~9', 'man_10~14', 'man_15~19', 'man_20~24', 'man_25~29',
'man_30~34', 'man_35~39', 'man_40~44', 'man_45~49', 'man_50~54', 'man_55~59',
'man_60~64', 'man_65~69', 'man_70~74', 'man_75~79', 'man_80~84', 'man_85~89',
'man_90~94', 'man_95~99', 'man_100 이상',
'woman_0~4', 'woman_5~9', 'woman_10~14', 'woman_15~19', 'woman_20~24',
'woman_25~29', 'woman_30~34', 'woman_35~39', 'woman_40~44', 'woman_45~49',
'woman_50~54', 'woman_55~59', 'woman_60~64', 'woman_65~69', 'woman_70~74',
'woman_75~79', 'woman_80~84', 'woman_85~89', 'woman_90~94', 'woman_95~99',
'woman_100 이상',
'gender']

def load_data(filename):  # 필요에 따라 레이블을 사용할수도 안할수도 있음
    df = pd.read_csv(filename, encoding="utf-8-sig")[colnames]

    X = df.loc[:, colnames[1:]].values
    y = df.loc[:, colnames[0]].values  # 레이블 값을 지난번과 다르게 0~8까지로 사용하도록 변경

    return X, y

X, y = load_data("result4_test.csv")
print(X.shape, y.shape)

(12444, 125) (12444,)


In [7]:
y.min(), y.max()

model = Sequential()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)
checkpoint = tf.keras.callbacks.ModelCheckpoint("lstm01.h5", monitor="val_loss", verbose=1 , save_best_only=True, mode="auto")

Adam = tf.keras.optimizers.Adam(clipnorm=1.)

model = Sequential()

model.add(LSTM(512, input_shape=(X.shape[1], 1), return_sequences=True)) #input_shape은 x의 라벨값 6개 시퀀스 출력은 True 512차원
model.add(Dropout(0.5)) #과적합 방지를 위한 드랍아웃 비율은 0.5
model.add(LSTM(256, return_sequences=True)) #LSTM 층  256차원출력
model.add(Dropout(0.5)) #과적합 방지를 위한 드랍아웃 비율은 0.5
model.add(LSTM(128, return_sequences=True)) #LSTM 층  256차원출력+
model.add(Dropout(0.5)) #드랍아웃 층
model.add(LSTM(64, return_sequences=True)) #LSTM 층  256차원출력+
model.add(Dropout(0.5)) #드랍아웃 층

model.add(LSTM(32)) #LSTM층 128차원 출력
model.add(Dropout(0.5)) #과적합 방지를 위한 드랍아웃 비율은 0.5

model.add(Dense(100)) #활성화 함수
model.add(Flatten())
model.add(Dense(1, activation='relu')) #활성화 함수

model.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mse'])
model.summary()

model.fit(X, y, epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping, checkpoint])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_19 (LSTM)              (None, 125, 512)          1052672   
                                                                 
 dropout_19 (Dropout)        (None, 125, 512)          0         
                                                                 
 lstm_20 (LSTM)              (None, 125, 256)          787456    
                                                                 
 dropout_20 (Dropout)        (None, 125, 256)          0         
                                                                 
 lstm_21 (LSTM)              (None, 125, 128)          197120    
                                                                 
 dropout_21 (Dropout)        (None, 125, 128)          0         
                                                                 
 lstm_22 (LSTM)              (None, 125, 64)         

2022-08-08 15:14:06.426489: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 15:14:06.967511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 15:14:07.185702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 15:14:07.326474: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 15:14:07.474072: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 15:14:07.630351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-08 15:14:07.827104: I tensorflow/core/grappler/optimizers/cust

389/389 [==============================] - 65s 151ms/step - loss: nan - mse: nan
Epoch 2/100
389/389 [==============================] - 57s 145ms/step - loss: nan - mse: nan
Epoch 3/100
389/389 [==============================] - 57s 145ms/step - loss: nan - mse: nan
Epoch 4/100
389/389 [==============================] - 57s 147ms/step - loss: nan - mse: nan
Epoch 5/100
389/389 [==============================] - 57s 147ms/step - loss: nan - mse: nan
Epoch 6/100
389/389 [==============================] - 57s 147ms/step - loss: nan - mse: nan
Epoch 7/100
382/389 [============================>.] - ETA: 1s - loss: nan - mse: nan

KeyboardInterrupt: 

In [ ]:
model.save("./pknu_bh.h5")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# X, y = load_data("/content/drive/MyDrive/wheather/data/only78.csv")
test_df = pd.read_csv("/content/drive/MyDrive/wheather/test/only78.csv")[colnames].astype(np.float32)
X = test_df.loc[:, colnames[1:]].values
y = test_df.loc[:, colnames[0]].values

y_score = model.predict(X)
print("MSE: " + str(mean_squared_error(y, y_score)))
print("RMSE: " + str(np.sqrt(mean_squared_error(y, y_score))))
print("MAE: " + str(mean_absolute_error(y, y_score)))
# y_pred = np.argmax(y_score, axis=1)
# pd.Series(y_pred, name=check_col[0]).to_csv("LSTM_result.csv")

In [ ]:
import pandas as pd


extract_col = ['YearMonthDayHourMinute', 'isitu-LST','insitu-TG','Band1','Band2','Band3','Band4','Band6','Band7','Band8','Band9','Band10','Band11','Band12','Band13','Band14','Band15','Band16','30daysBand3','30daysBand13','SolarZA','SateZA','ESR','Height','LandType','insitu-HM','insitu-TD']

df = pd.read_csv("/content/drive/MyDrive/wheather/data/과제2 결측치 제거_3.2g.csv")[extract_col]
df['month'] = pd.to_datetime(df['YearMonthDayHourMinute'], format='%Y%m%d%H%M%S').dt.strftime('%m').astype(int)
extract_col.append('month')

# df['LandType'] += 1

df.to_csv("/content/drive/MyDrive/wheather/data/과제2 결측치 제거_3.2g_2.csv")